# Assignment 2: The Winter is here
##### This works best with epic battle music. No spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
import torchvision
import random
from collections import namedtuple
from PIL import Image


%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire

In [2]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
resnet_input = 224 
batch_size = 2
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [3]:
classes = ('__background__', 'aeroplane', 'bicycle', 'car', 'cat', 'dog')

# counts to maintain the number of training examples in each of the 6 classes
counts = [0]*6

In [4]:
# Function to create the directories corresponding to images of each of 6 classes

def create_hierarchical_dirs():
    if not os.path.exists('data/train'):
        os.makedirs('data/train')      
        
    for i in range(len(classes)):
        dirname = 'data/train/' + classes[i]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

create_hierarchical_dirs()

In [5]:
# Function to crop and distribute the given training data images in their corresponding class directories 

def jamie_bronn_build_dataset():
    
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')    
    for img_name in dir_names:
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                for xmin in object1.iter('xmin'):
                    x1 = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    x2 = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    y1 = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    y2 = int(float(ymax.text))

                img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
                img = img.crop((x1,y1,x2,y2)).resize((resnet_input,resnet_input))
                if dirname in classes:
                    img.save('data/train/' + dirname + '/' + str(counts[classes.index(dirname)]) + ".jpg");
                    counts[classes.index(dirname)]+=1
                    
jamie_bronn_build_dataset()

In [6]:
# Function to make a rectangle using four coordinates (x1,y1,x2,y2)

Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

# Function to calculate the overlappig area of 2 given rectangles

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy


In [7]:
# Function to create negative samples i.e. background class images

def build_background_class():
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    average = int(sum(counts[1:])/len(counts[1:]))
    counts[0] = 0
    while(counts[0]!=average): 
        select = 1
        img_name = random.choice(dir_names)
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
        imsize = img.size;
        x1 = random.randrange(imsize[0])
        x2 = random.randrange(imsize[0])
        y1 = random.randrange(imsize[1])
        y2 = random.randrange(imsize[1])
        if(x1>x2):
            temp = x1
            x1=x2
            x2=temp
        if(y1>y2):
            temp = y1
            y1=y2
            y2=temp
        if(x1==x2 or y1==y2):
            continue
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                intersection = 0
                for xmin in object1.iter('xmin'):
                    xa = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    xb = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    ya = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    yb = int(float(ymax.text))
                ra = Rectangle(x1, x2, y1, y2)
                rb = Rectangle(xa, xb, ya, yb)
                if(area(ra,rb)!=None):
                    intersection = area(ra,rb)
                union = ((x2-x1)*(y2-y1))+((xb-xa)*(yb-ya))-intersection
                if(float(intersection)/union>0.5):
                    select = 0

        if(select==1):
            img = img.crop((x1,y1,x2,y2)).resize((resnet_input,resnet_input))
            img.save('data/train/__background__/' + str(counts[0]) + ".jpg");
            counts[0] += 1
    
build_background_class()

In [8]:
# Function to build train_loader ad test_loader

# counts to maintain the number of training examples in each of the 6 classes
counts = [0]*6

class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        
        
        if (train==True): 
            
            # In case of train_loader, return images and their corresponding labels
            
            dir1 = '/train/'
            dir_names = os.listdir(root_dir + dir1)
            img_names=[]
            labels = []
            count = 0
            for temp in range(len(dir_names)):
                names = os.listdir(root_dir + dir1 + classes[temp])
                N = len(names)
                for n in range(N):
                    img_names.append(str(root_dir + dir1 + classes[temp] + '/' + names[n]))
                    labels += [temp]
                    counts[temp] += 1
            self.labels = labels 
         
        
        
        else:
            
            # In case of test_loader, return images and their corresponding xml file_names. These xml_names are later
            # used to extract the ground-truth bouning boxes and calculate the accuracy of the model
            
            img_names=[]
            xml_names=[]
            dir1 = '/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages'
            dir2 = '/VOCtest_06-Nov-2007/VOCdevkit/VOC2007/Annotations'
            dir_names = os.listdir(root_dir + dir1)
            for name in dir_names:
                img_names.append(str(root_dir + dir1 + '/' + name))
                xml_names.append(str(root_dir + dir2 + '/' + name[:-4] + '.xml'))
            self.xml_names = xml_names
                
        self.img_names = img_names    
        self.train = train
        self.root_dir = root_dir
        self.transform = transform
        
        
                
        
        
        
    def __len__(self):
        # Begin
        
        return len(self.img_names)        
        
    def __getitem__(self, idx):
        # Begin      
        
        img_pil = Image.open(self.img_names[idx])
        if(self.train==True):
            # In case of train_loader, return images and their corresponding labels            
            return self.transform(img_pil), self.labels[idx]
        else:
            # In case of test_loader, return images and their corresponding xml file_names. These xml_names are later
            # used to extract the ground-truth bouning boxes and calculate the accuracy of the model            
            return self.transform(img_pil), self.xml_names[idx]     
        

## Train the network
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [9]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.RandomHorizontalFlip(),transforms.ToTensor()])
train_dataset = hound_dataset(root_dir='./data', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = hound_dataset(root_dir='./data', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [10]:
resnet18 = models.resnet18(pretrained=True)

for param in resnet18.parameters():
    param.requires_grad = False

resnet18.fc = nn.Linear(resnet18.fc.in_features, 6)
resnet18.cuda()

# Add code fors using CUDA here

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): BasicBlock (
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d

In [11]:
# Creating weights for each of the 6 classes to account for the imbalance in the amount of training examples 
# of each of these classes

weights = [0]*6
total_images = sum(counts)
for i in range(len(weights)):
    weights[i] = total_images/counts[i]
weights = torch.FloatTensor(weights)

criterion = nn.CrossEntropyLoss(weight=weights).cuda()
optimizer = optim.SGD(resnet18.fc.parameters(), learning_rate, hyp_momentum)

In [12]:
# training the model

def arya_train():
    # Begin
    for epoch in range(num_epochs): 
        resnet18.train()        
        for i, (input1,target) in enumerate(train_loader):
            
            input_var = torch.autograd.Variable(input1).cuda()            
            target = target.cuda(async = True)
            target_var = torch.autograd.Variable(target)
            
            output = resnet18(input_var)
            loss = criterion(output, target_var)           
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            output = output.float()
            loss = loss.float()

In [13]:
%time arya_train()

CPU times: user 7min 6s, sys: 1min 39s, total: 8min 46s
Wall time: 4min 25s


In [14]:
# Saving the trained model parameters

torch.save(resnet18.state_dict(), 'params.pkl')
#resnet18.load_state_dict(torch.load('params.pkl'))

In [15]:
# Calculating the training accuracy

def train_accuracy():
    correct = 0
    total = 0
    
    for i, (input1,target) in enumerate(train_loader):
            
            input_var = torch.autograd.Variable(input1,volatile = True).cuda()
            target = target.cuda(async = True)
            target_var = torch.autograd.Variable(target)
                        
            output = resnet18(input_var)
            output = output.float()
            
            # t1 gives the array of predicted classes and t2 is the array of original class 
            t1 = output.max(1)[1].cpu().data.numpy()
            t2 = target_var.cpu().data.numpy()
            
            # Updating both the total and correct by adding the total no. of images in this batch as well as the
            # correctly classified images in this batch
            correct += sum(t1==t2)
            total += len(t2)
            
    print('Train Accuracy: '+str(correct*100.0/total))

In [16]:
%time train_accuracy()

Train Accuracy: 68.1726907631
CPU times: user 1min 39s, sys: 28.5 s, total: 2min 8s
Wall time: 1min 4s


# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [17]:
def theon_sliding_window():
    # Generating a range of sliding windows using 3 different sizes and 3 different aspect ratios
    # and returning the corresponding rectangle coordinates
    
    boxes=[]
    imres = [[192,192],[192,96],[96,192],[128,128],[128,64],[64,128],[64,64],[64,32],[32,64]]
    for i in range(0,resnet_input,10):
        for j in range(0,resnet_input,10):
            for window in imres:
                boxes.append([i-window[0]/2, j-window[1]/2, i+window[0]/2, j+window[1]/2])
    return np.asarray(np.array(boxes))

sliding_windows = theon_sliding_window()
print('Total number of sliding windows per image:  ' + str(len(sliding_windows)))

Total number of sliding windows per image:  4761


In [18]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.7):
        
        boxes = np.array(boxes)
        final_boxes = []

        # Extracting th x1,y1,x2,y2 co-ordinates from the boes array
        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]

        # Calculating area for each of the (x1,y1,x2,y2) tuple in the boxes list
        area = (x2 - x1 + 1) * (y2 - y1 + 1)
        idxs = np.argsort(y2)
        
        
        # Selecting a box and deleting boxes in the list that have greater than some threshold overlap 
        # with the selected box
        
        while len(idxs) > 0:
            
            last = len(idxs) - 1
            i = idxs[last]
            final_boxes.append(i)
            suppress = [last]
            
            for pos in range(0, last):
                
                j = idxs[pos]
                
                ra = Rectangle(x1[i],y1[i],x2[i],y2[i])
                rb = Rectangle(x1[j],y1[j],x2[j],y2[j])
                intersection = area(ra,rb)
                
                if(area(ra,rb) == None):
                    overlap = 0
                else:
                    overlap = intersection / area[j]

                if overlap > threshold:
                    suppress.append(pos)

            idxs = np.delete(idxs, suppress)

        return boxes[final_boxes]
        

"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [19]:
# Function to return the list of ground-truth bounding boxes in an image given its xml file_name

def retrieve_bbox_list(xml):
    
    bbox_list = []
    tree = ET.parse(xml)
    root = tree.getroot()
    for object1 in root.iter('object'):
        for name1 in object1.iter('name'):
            classname = name1.text                        
        for bndbox in object1.iter('bndbox'):
            for xmin in object1.iter('xmin'):
                x1 = int(float(xmin.text))
            for xmax in object1.iter('xmax'):
                x2 = int(float(xmax.text))
            for ymin in object1.iter('ymin'):
                y1 = int(float(ymin.text))
            for ymax in object1.iter('ymax'):
                y2 = int(float(ymax.text))
            if(classes.__contains__(classname)):
                bbox_list.append([x1,y1,x2,y2,classes.index(classname)])  
    return bbox_list

In [20]:
# Function to test the accuracy of the model

temp = np.empty((3,224,224))
def daenerys_test():
    
    total = 0
    correct = 0
    
    transform_PIL = transforms.Compose([torchvision.transforms.ToPILImage()])
    
    for i, (inputvar ,xml_names) in enumerate(test_loader):
            
            # Iterating on each image in the given batch and extracting the cropped versions of each image corresponding
            # to each sliding window, then feeding each of these into the network and appyling the mean-shift-clustering 
            # technique to obtain the final detection windows
            
            for iter1 in range(len(inputvar)):
                
                temp = inputvar[iter1]
                img = transform_PIL(inputvar[iter1])
                xml = xml_names[iter1]
                
                gd_bbox_list = retrieve_bbox_list(xml)
                boxes = []
                window_count = 0
                
                # Iterating over each sliding window and passing the cropped versions of the image to the conv-net to
                # obtain the class-probabilities
                
                for window in sliding_windows:
                    window_count += 1
                    input_conv = composed_transform(img.crop(window))
                    input_var = torch.autograd.Variable(input_conv, volatile = True).cuda()
                    input_var = input_var.unsqueeze(0)
                    output = resnet18(input_var)
                    output = output.float()
                    pred = output.max(1)[1].cpu().data.numpy()                    
                    pred_prob = output.max(1)[0].cpu().data.numpy() 
                    
                    boxes.append([window[0],window[1],window[2],window[3],pred_prob[0],pred[0]])
                 
                
                # Identifying the boxes having high probablility of containing an object(of any class)
                
                new_boxes = []
                for box in boxes:
                    if box[4]>0.3 and box[5]!=0:
                        new_boxes.append(box)
                boxes = new_boxes
                
                # Applying the non-maximal-suppression to obtain the final detection windows
                pred_bboxes = aegon_targaryen_non_maximum_supression(boxes)
                
                # Calculating the accuracy of the model by calculating the number of predicted windows which have 
                # intersection over union greater than 0.5 with the ground truth windows and have the same class
                total += len(gd_bbox_list)
                correct_bool = [0]*total
                for pred_bbox in pred_bboxes:
                    for gd_bbox in gd_bbox_list:
                        ra = Rectangle(pred_bbox[0], pred_bbox[1], pred_bbox[2], pred_bbox[3])
                        rb = Rectangle(gd_bbox[0], gd_bbox[1], gd_bbox[2], gd_bbox[3])
                        if(area(ra,rb)==None):
                            continue
                        union = (gd_bbox[3]-gd_bbox[1])*(gd_bbox[2]-gd_bbox[0]) + \
                                (pred_bbox[3]- pred_bbox[1])*(pred_bbox[2]- pred_bbox[0])- area(ra,rb)
                        print(pred_bbox[5],gd_bbox[4])
                        if (area(ra, rb) > 0.5*union and pred_bbox[5] == gd_bbox[4]):
                            correct_bool[gd_bbox_list.index(gd_bbox)] = 1
                correct += sum(correct_bool)
                
     
    accuracy = correct/total * 100
                
    print('Test Accuracy: ' + str(accuracy))  

In [ ]:
# NOTE: Number of test images is approx. 5K. and for each image, we have to extract and process 4761 sliding windows
# and this is taking a lot of time

%time daenerys_test()

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"